In [1]:
import pdfplumber
import ollama
import os
import json

path = "./vidhya_LFT.pdf"
json_string=""
filename_without_ext=os.path.splitext(os.path.basename("hello"))[0]
json_filename = f"{filename_without_ext}.json"

def pdf_to_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

result = pdf_to_text(path)

In [2]:
response = ollama.chat(
  model="llama3.1:8b",
  messages=[
    {
      "role": "user",
      "content": result + """\n\nthis is a patient's medical report. Read through the entire data and rewrite 
                    it in such a way that the data is more user-readable but detailed, without changing the values and names of the tests, 
                    departments, and others. Return a JSON document with data. Only return JSON, no other text."""
    }
  ],
)

json_string = response['message']['content']

In [3]:
print(json_string)

```json
{
  "patient": {
    "name": "MS. VIDHYA.T",
    "sid_no": "08016608",
    "age_sex": "38 Y / Female"
  },
  "registration": {
    "date_time": "28/05/2018 07:50:47",
    "referrer": "Dr. PROF.CHANDRAMOHAN.S.M MB.,MS.,MCH.,FACS.",
    "branch": "PORUR"
  },
  "investigation": [
    {
      "department": "BIOCHEMISTRY",
      "test_name": "ELECTROLYTES",
      "sodium": {
        "result": "136.1 mmol / L",
        "units": "mmol/L",
        "biological_reference_interval": "137 - 145 mmol / L"
      }
    },
    {
      "department": "BIOCHEMISTRY",
      "test_name": "POTASSIUM",
      "result": "4.2 mmol / L",
      "units": "mmol/L",
      "biological_reference_interval": "3.4 - 5.1 mmol / L"
    },
    {
      "department": "BIOCHEMISTRY",
      "test_name": "CHLORIDE",
      "result": "101.5 mmol / L",
      "units": "mmol/L",
      "biological_reference_interval": "98 - 107 mmol / L"
    },
    {
      "department": "BIOCHEMISTRY",
      "test_name": "BICARBONATE",
      

In [5]:
json_string = json_string.replace("json\n","").replace("\n","").replace("```","")
json_data = json.loads(json_string)
  
with open(json_filename,'a') as file:
    json.dump(json_data,file,indent=4)

In [15]:
f= open(json_filename)
temp_data= str(json.loads(f.read()))

response1 = ollama.chat(
  model="llama3.1:8b",    
  messages=[
    {
      "role": "user",
      "content": temp_data + """\n play the role of a doctor, this is a patient's medical report. go thorugh the entire data 
          and give me only the abnormal parameters(a parameter is considered abnormal if the patient's result value doesn't fall inside the normal
          range / reference interval mentioned in the report), as individual bullet points and no other text"""
    }
  ],
)

jstring = response1['message']['content']
print(jstring)
f.close()

• Sodium: 136.1 mmol / L (normal reference interval: 137 - 145 mmol / L)
• Potassium: Not abnormal (4.2 mmol / L within normal reference interval of 3.4 - 5.1 mmol / L)
• Chloride: Not abnormal (101.5 mmol / L within normal reference interval of 98 - 107 mmol / L)
• Bicarbonate: Not abnormal (23.0 mmol / L within normal reference interval of 22 - 30 mmol / L)
• Glucose (F): Not abnormal (89.0 mg/dl within normal reference interval of 70-110 mg/dL)
• Bilirubin total: 2.54 mg/dl (normal reference interval: 0.0-2.0 mg/dl)
• Bilirubin direct: 1.72 mg/dl (normal reference interval: 0.1 - 0.40 mg/dl)
• Bilirubin indirect: 0.82 mg/dl (within normal reference interval of 0.2 - 0.9 mg/dl, note: it's a bit at the lower limit)
• SGOT(AST): 359.0 U/L (normal reference interval: <31 U/L)
• SGPT(ALT): 431.0 U/L (normal reference interval: <34 U/L)
• Alkaline phosphatase: Not abnormal (173.0 U/L within normal reference interval of 42-98 U/L)
• Gamma GT: 513.0 U/L (within the normal reference interval